In [1]:
%cd D:\YeqinHUANG_Collection\ucl\week14_28.Jan.2024\06text\epub

D:\YeqinHUANG_Collection\ucl\week14_28.Jan.2024\06text\epub


In [2]:
import ebooklib
from ebooklib import epub
import re
import os

In [28]:
def read_epub_book(epub_file,ID):
    book = epub.read_epub(epub_file)
    text = ''
    
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?','',content) #remove the html
            content = re.sub('\s+',' ',content) #remove the black
            content = re.sub('\n',' ',content)#remove the new line
            content = re.sub("&#13;", ' ', content)# remove the enter
            
            text += content
            
    print(content)

In [29]:
def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []

    for s in strings:
        if len(merged_string) <= min_length:# and len(s)<max_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
            splitParagraph = merged_string.split('.')
            splitParagraphRePoint = []
            for sp in splitParagraph:
                splitParagraphRePoint.append(sp+'.')
            merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
            merged_strings.extend(merged)
            merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s

    if merged_string:
        merged_strings.append(merged_string)

    return merged_strings

In [30]:
def read_epub_paragraphs(epub_file, ID):
    book = epub.read_epub(epub_file)
    paragraphs = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            paragraphs.extend(content.strip().split("&#13;"))
    paragraphs = merge_strings_until_limit(paragraphs, 200, 1000)
    paragraphs = [{'paragraph':paragraphs[i], 'nr':i,'bookID':ID} for i in range(len(paragraphs)) if len(paragraphs[i])>50]
    return paragraphs[1:-1]

In [31]:
import os

def process_epub_files_in_folder(folder_path):
    epub_files = [file for file in os.listdir(folder_path) if file.endswith(".epub")]
    all_paragraphs = []  
    
    for epub_file in epub_files:
        epub_path = os.path.join(folder_path, epub_file)
        paragraphs = read_epub_paragraphs(epub_path, epub_file)
        all_paragraphs.extend(paragraphs)  
        
    return all_paragraphs  

folder_path = r"D:\YeqinHUANG_Collection\ucl\week14_28.Jan.2024\06text\epub"
all_paragraphs = process_epub_files_in_folder(folder_path)



C:\Users\76392\anaconda3\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [32]:
all_paragraphs

[{'paragraph': '--death of Valentinian Ii.--second Civil War, Against Eugenius.-- Death of Theodosius. Part II: Part III: Part IV: Part V: Chapter XXVIII Part I: Final Destruction of Paganism.--introduction of the Worship of Saints, and Relics, Among the Christians.',
  'nr': 1,
  'bookID': 'gibbon-history-of-the-decline-and-fall-of-the-roman-empire-volume-3.epub'},
 {'paragraph': ' Part II: Part III: Chapter XXIX Part I: Final Division of the Roman Empire Between the Sons of Theodosius.--reign of Arcadius and Honorius-- Administration of Rufinus and Stilicho.--revolt and Defeat of Gildo in Africa.',
  'nr': 2,
  'bookID': 'gibbon-history-of-the-decline-and-fall-of-the-roman-empire-volume-3.epub'},
 {'paragraph': ' Part II: Chapter XXX Part I: Revolt of the Goths.--they Plunder Greece.--two Great Invasions of Italy by Alaric and Radagaisus.--they Are Repulsed by Stilicho.--the Germans Overrun Gaul.--usurpation of Constantine in the West.',
  'nr': 3,
  'bookID': 'gibbon-history-of-the-

In [33]:
texts = []

for paragraph_data in all_paragraphs:
    paragraph_texts = paragraph_data['paragraph']
    texts.append(paragraph_texts)

In [34]:
texts

['--death of Valentinian Ii.--second Civil War, Against Eugenius.-- Death of Theodosius. Part II: Part III: Part IV: Part V: Chapter XXVIII Part I: Final Destruction of Paganism.--introduction of the Worship of Saints, and Relics, Among the Christians.',
 ' Part II: Part III: Chapter XXIX Part I: Final Division of the Roman Empire Between the Sons of Theodosius.--reign of Arcadius and Honorius-- Administration of Rufinus and Stilicho.--revolt and Defeat of Gildo in Africa.',
 ' Part II: Chapter XXX Part I: Revolt of the Goths.--they Plunder Greece.--two Great Invasions of Italy by Alaric and Radagaisus.--they Are Repulsed by Stilicho.--the Germans Overrun Gaul.--usurpation of Constantine in the West.',
 '--disgrace and Death of Stilicho. Part II: Part III: Part IV: Part V: Chapter XXXI Part I: Invasion of Italy by Alaric.--manners of the Roman Senate and People.--rome is Thrice Besieged, and at Length Pillaged, by the Goths.',
 '--death of Alaric.--the Goths Evacuate Italy.--fall of Co

In [15]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
model = Doc2Vec(vector_size=250, min_count=0, alpha = 0.025, min_alpha=0.025, epochs=100)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [38]:
text_vector = model.infer_vector(["spain", "movement", "instructure", "internal", "crowds"])
similar_documents = model.docvecs.most_similar([text_vector], topn=5)

for doc_id, similarity in similar_documents:
    print(f"Document ID: {doc_id}, Similarity: {similarity}")
    print(texts[doc_id])
    print("------")


Document ID: 2739, Similarity: 0.21706795692443848
 As a Roman, he had been educated in the liberal studies of rhetoric and jurisprudence; but he was engaged by accident and policy in the familiar use of the Germanic idiom. The independent Barbarians resorted to the tribunal of a stranger, who possessed the singular talent of explaining, in their native tongue, the dictates of reason and equity.
------
Document ID: 2835, Similarity: 0.2075413465499878
 But the Romans were disposed to revere, in the person of their master, that antique title which the emperors condescended to assume: the Barbarian himself seemed to contract a sacred obligation to respect the majesty of the republic; and the successors of Theodosius, by soliciting his friendship, tacitly forgave, and almost ratified, the usurpation of Gaul.
------
Document ID: 307, Similarity: 0.20535936951637268
 They applauded his chastity and temperance, his contempt of pleasure, his application to business, and his tender affection f

C:\Users\76392\AppData\Local\Temp\ipykernel_33292\2390257245.py:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_documents = model.docvecs.most_similar([text_vector], topn=5)
